# Initialize Folders

In [6]:
from __future__ import print_function
from imutils import paths
import os
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import keras
from pathlib import Path

from keras.preprocessing import image as image_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import backend as keras

from datetime import datetime

from UnetModel import *
#import skimage.io as io
#import skimage.transform as trans

#K.set_image_data_format("channels_last")

%matplotlib inline
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
img_rows = 256
img_cols = 256

#Training data generation
data_gen_args = dict(
#    samplewise_center = True,
#    samplewise_std_normalization = True,
    rotation_range=180,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    vertical_flip = True,
    fill_mode='nearest')

#Validation data generation
data_val_gen_args = dict(
    #samplewise_center = True,
    #samplewise_std_normalization = True
    )

#Create UNet Model
#model = FullUnetModel()
#model = UnetModel()
#model = BiggerLeakyUnetModelWithBatchnorm()
model = BiggerLeakyUnetModel()
#model = BiggerLeakyUnetModelWithLayernorm()
#model = BiggerLeakyUnetModelWithBatchLayernorm()

#Setup generator
batch_size = 2

train_path = Path.home() / 'data/isic_2018/train'
val_path = Path.home() / 'data/isic_2018/val'
      
myGene = trainGenerator(batch_size,
                        str(train_path),
                        'images',
                        'masks',
                        data_gen_args, 
                        target_size= (img_rows,img_rows))
myValGene = validationGenerator(batch_size,
                                str(val_path),
                                'images',
                                'masks',
                                data_val_gen_args, 
                                target_size= (img_rows,img_rows))

#Create folder for models
date_object = datetime.now()
# convert object to the format we want
formatted_date = date_object.strftime('%Y%m%d')
output_dir = 'unet/{}'.format(formatted_date)
os.makedirs(output_dir, exist_ok =True)

#Setup Checkpoint to only capture best estimate
model_checkpoint = ModelCheckpoint('{}/unet_lesion_{{epoch:03d}}-{{val_jaccard_coef:.5f}}.hdf5'.format(output_dir)
                                   , monitor='val_jaccard_coef'
                                   ,verbose=1, mode='max', save_best_only=True)


/home/ilyarudyak/projects/cs230_project_keras/skin_lesion/example/UnetModel.py:482: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input=inputs, output=output)


In [10]:
iterations = 2 # 100
#Train
history = model.fit_generator(
    myGene,
    steps_per_epoch = 1000, 
    epochs=iterations,
    callbacks=[model_checkpoint],
    validation_data=myValGene,
    validation_steps=260)

model.save(os.path.join(output_dir, 'iter_{}'.format(iterations)))

Epoch 1/2
Found 519 images belonging to 1 classes.
Found 1556 images belonging to 1 classes.
Found 519 images belonging to 1 classes.
Found 1556 images belonging to 1 classes.
1000/1000 [==============================] - 712s 712ms/step - loss: -0.5701 - jaccard_coef: 0.5701 - val_loss: -0.6543 - val_jaccard_coef: 0.7072

Epoch 00001: val_jaccard_coef improved from -inf to 0.70721, saving model to unet/20191129/unet_lesion_001-0.70721.hdf5
Epoch 2/2
 147/1000 [===>..........................] - ETA: 5:24 - loss: -0.6743 - jaccard_coef: 0.6743

KeyboardInterrupt: 

In [ ]:
print(history.history['val_loss'])

In [ ]:
model.summary()

In [ ]:
#Continue traing
#Use initial_epoch 

history2 = model.fit_generator(
    myGene,
    steps_per_epoch = 1000, 
    epochs=200,
    callbacks=[model_checkpoint,tensorBoard], 
    initial_epoch = 100,
    validation_data=myValGene,
    validation_steps=100)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['jaccard_coef'])
plt.plot(history.history['val_jaccard_coef'])
plt.title('Jaccard Index')
plt.ylabel('Jaccard Index')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.legend(['Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.legend(['Validation'], loc='upper left')
plt.show()

In [ ]:
model.save('unet_lesion_20190520_batchnorm_100_epochs.hdf5')

In [ ]:
#This code combined 2 seperate training history together and plot the result
hist = {}
for i in history.history.keys():
    hist_concate = np.array([np.array(history.history[i]), np.array(history2.history[i])]).flatten() 
    hist[i] = hist_concate
# Plot training & validation accuracy values
plt.plot(hist['jaccard_coef'])
plt.plot(hist['val_jaccard_coef'])
plt.title('Coefficiency')
plt.ylabel('Coefficiency')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.legend(['Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.legend(['Validation'], loc='upper left')
plt.show()

coef = np.array(hist['jaccard_coef'])
val_coef = np.array(hist['val_jaccard_coef'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))


In [ ]:
coef = np.array(history.history['jaccard_coef'])
val_coef = np.array(history.history['val_jaccard_coef'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))


In [ ]:
model.load_weights("unet_lesion_20190510_0.844.hdf5")

In [ ]:
model.load_weights("unet/unet_lesion_20190509_0.82.hdf5")

file_names = next(os.walk(test_data_dir))[2]
scores = []
for file in file_names:
    grey_img = load_img(os.path.join(test_data_dir,file), target_size=(img_rows, img_cols), grayscale=False)
    mask_img = load_img(os.path.join(test_data_mask_dir,file.split('.')[0]+"_segmentation.png"), 
                        target_size=(img_rows, img_cols), grayscale=True)
    img = img_to_array(grey_img)
    img_mask = img_to_array(mask_img)
    
    #Preprocess image mask
    #img_mask = img_mask /255
    #img_mask[img_mask > 0.5] = 1
    #img_mask[img_mask <= 0.5] = 0
    #Preprocess images
    #mean = np.mean(img)  # mean for data centering
    #std = np.std(img)  # std for data normalization
    #img -= mean
    #img /= std
    img, img_mask = normalizeData(img, img_mask)
    img = np.reshape(img,(1,)+img.shape)
    
    
    
    pred = model.predict([img])
    sess = tf.Session()
    score = sess.run(jaccard_coef(img_mask, pred))
    print("{} -- jaccard index: {}".format(file,score))
    scores.append([file,score])

    result_img = array_to_img(pred[0] * 255 )
    result_img.save(os.path.join(test_data_pred_dir, file.split('.')[0] + '_predict.jpg'))

with open("unet_test_result.csv", 'w') as f:
    f.write("filename, jaccard_index\n")
    for i in range(len(scores)):
        #print(scores[i])
        f.write("{},{}\n".format(scores[i][0], scores[i][1]))